In [17]:
import torch
from model import load_resnet_model
import os
from train_regression_weighted_loss import train_model
import re
from dense_weight import DenseWeight
from dataloader import create_dataloader
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_resnet_model('resnet50', num_classes=1)

c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lhmtr\OneDrive\Desktop\remote-sensing-asset-index\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
def get_file_paths(directory):
    """
    Get all file paths in the specified directory with the specified file extension.

    Args:
    - directory (str): The directory to search for files.
    - file_extension (str): The file extension to filter by.

    Returns:
    - List[str]: A list of file paths.
    """
    file_paths = []
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        file_paths.append(file_path)
    return file_paths

# Use a regular expression to extract the batch number from the filename
def extract_batch_number(file_path):
    match = re.search(r"data_batch_(\d+)", file_path)
    if match:
        return int(match.group(1))
    else:
        return -1  # If for some reason a file doesn't match the pattern


In [13]:
train_files = sorted(get_file_paths('./data/records/train/'), key= extract_batch_number)
val_files = sorted(get_file_paths('./data/records/val/'), key= extract_batch_number)

In [ ]:
training_labels = []

for train_file in train_files:
    train_loader = create_dataloader(train_file, True, 32)
    for _, labels in tqdm(train_loader):
        training_labels.extend(labels.detach().cpu().numpy())

dense_weight_model = DenseWeight(0.5)
dense_weight_model.fit(training_labels)

In [5]:
train_model(model, train_files, val_files, device, dense_weight_model,epochs=15)

100%|██████████| 3/3 [00:01<00:00,  2.89it/s]


Epoch 1/10, Train Loss: 1.0007, Val Loss: 1.2756, F1 Macro: 0.2628, F1 Micro: 0.5494
Model saved as checkpoint_epoch_1.pth


100%|██████████| 3/3 [00:00<00:00,  3.39it/s]


Epoch 2/10, Train Loss: 0.8812, Val Loss: 1.5962, F1 Macro: 0.2859, F1 Micro: 0.5023


100%|██████████| 3/3 [00:00<00:00,  3.31it/s]


...

Epoch 8/10, Train Loss: 0.7365, Val Loss: 0.9052, F1 Macro: 0.4246, F1 Micro: 0.6299
Model saved as checkpoint_epoch_8.pth


100%|██████████| 3/3 [00:00<00:00,  3.31it/s]


Epoch 9/10, Train Loss: 0.7045, Val Loss: 1.1353, F1 Macro: 0.3708, F1 Micro: 0.5732


100%|██████████| 3/3 [00:00<00:00,  3.31it/s]


Epoch 10/10, Train Loss: 0.6635, Val Loss: 1.2270, F1 Macro: 0.3441, F1 Micro: 0.5499
Model saved as final_model.pth
Training completed and final model saved.
